general prep of data.

In [1]:
#if jupyternotify is installed, we can add %notify to a cell to get an alert when it ifnished running

%load_ext jupyternotify
import metrics_helpers as indicators
import pickle as pk
import gc
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd


def dt_to_int(dt): #datetime to integer
    return dt.astype('int')/(10**9)
with open('/Users/sma/Documents/INRAE internship/scrape-git/netmums/allposts_rerun.pkl', 'rb') as f:
    netmums = pk.load(f)
#with open('/Users/sma/Documents/INRAE internship/scrape-git/netmums/netmums_subset_keys.txt', 'r') as f:
#    keys = [url.strip() for url in f.readlines()]
    
    
#netmums = {key:netmums[key] for key in keys}

<IPython.core.display.Javascript object>

In [2]:
#netmums
nm_ind = indicators.indicators(netmums, fb=False)
#this one takes long, around 20 seconds I think.

nm_ind.add_available_comments()
nm_ind.add_num_unique_posters()
nm_ind.add_num_urls()
nm_ind.add_post_time()
nm_ind.add_lexical_richness()
nm_ind.add_term_distance_simple()

netmums = nm_ind.results_dict

# GenSim / Word2Vec Implementation on Threads

https://radimrehurek.com/gensim/models/word2vec.html

In [3]:
from gensim.models import Word2Vec, Phrases

In [4]:
from gensim.models.phrases import ENGLISH_CONNECTOR_WORDS

we can try running the model on only our subset and see how long it takes.

Then maybe we can run it on the entire dataset.

The end goal of this is to determine which words are related to ngrams like "baby food"

In [5]:
#Process our data to the right format for feeding into the model
keys = list(nm_ind.text_dict.keys())
text_list = list(nm_ind.text_dict.values())


In [6]:
import re

In [7]:
#define functions
def deEmojify(text):
    #remove emoji (FIXME: doesnt remove all of them.)
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def clean(text):
    #remove URLs
    pdf_regex='http[\S]+pdf[\S]*'
    regex = r'http\S+'
    text = re.sub(regex, 'urlpostedtopdf', text)
    text = re.sub(regex, 'urlpostedtosomething', text)
    #TODO: remove emails

    #replace commas and semicolons with spaces.
    text = re.sub('[;,&\+]+', ' ', text)
    #remove hyphens
    text = re.sub('[-]+', ' ', text)
    return text

In [8]:
#CLEAN TEXT
text_list = [deEmojify(i) for i in text_list]

text_list = [clean(item) for item in text_list]


#split sentences, new lines
text_list = [j for i in text_list for j in re.split('[\n?!.]+', i)]

#remove empty items
text_list = [i for i in text_list if i]
#strip reamining elements from text
text_list = [re.sub(r'[^A-Za-z0-9 ]+', '', i) for i in text_list]
#remove extra spaces
text_list = [re.sub(r'\s+', ' ', i) for i in text_list]

In [9]:
from gensim.utils import tokenize

In [10]:
#we need to transform list of sentence to LoL of words
tokens = [list(tokenize(doc, lower=True)) for doc in text_list]

In [11]:
# Train a bigram detector.
#this detects bigrams and converts them to single tokens simply by relplacing space w underscore
#check the google paper for more info on how it's done.
#https://datascience.stackexchange.com/questions/25524/how-does-phrases-in-gensim-work

#Detect phrases based on collocation counts.
bigram_transformer = Phrases(tokens, threshold = 2, connector_words=ENGLISH_CONNECTOR_WORDS)
#moving the threshhold a bit higher.. about 12, will get rid of a lot
#of non-phrase bigrams (what_brand, an_email) but I'm not sure how this would be beneficial.
#TODO: read how the word similarity is calculated.

# Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
model = Word2Vec(bigram_transformer[tokens], min_count=1)

%notify

<IPython.core.display.Javascript object>

In [12]:
#score_candidate doesnt seem to work so we calculate it by ourselves
#for some reason baby_food is not being detected in phrases despite there being over 1000 occurences of it.
#are these occurences already in teh data?
bigram_transformer.scoring(
            worda_count=bigram_transformer.vocab.get('baby'),
            wordb_count=bigram_transformer.vocab.get('food'),
            bigram_count=bigram_transformer.vocab.get('baby_food'),
            len_vocab=len(bigram_transformer.vocab),
            min_count=bigram_transformer.min_count,
            corpus_word_count=bigram_transformer.corpus_word_count,
        )

4.23432557658802

In [13]:
bigram_transformer.vocab.get('jarred_food') #there's only 60 instances of the phrase "jarred food"

60

In [14]:
#bigram_transformer = Phrases(tokens, threshold = 10) 
temp = sorted(bigram_transformer.export_phrases().items(), key= lambda x:x[1] * -1 )

In [15]:
baby_ = [item for item in bigram_transformer.export_phrases().items() if 'baby_' in item[0]]
baby_

[('baby_to_cry', 2.651880858554233),
 ('baby_lotion', 5.128990937088103),
 ('baby_oil', 2.3506088536116336),
 ('baby_wipes', 11.565592750444821),
 ('baby_girl', 6.27950622935893),
 ('baby_rice', 19.550999973950216),
 ('baby_gros', 48.643522450400276),
 ('baby_shampoo', 5.715161329898172),
 ('baby_dust', 24.76236555213527),
 ('baby_steps', 2.1650789628832707),
 ('baby_bath', 2.8570263791849917),
 ('baby_sleep', 2.2151927927465183),
 ('baby_boy', 4.865474615193476),
 ('baby_massage', 27.168232026988306),
 ('baby_food', 4.23432557658802),
 ('baby_porridge', 8.080941815826687),
 ('baby_led', 50.0929770403221),
 ('baby_talc', 3.417106948995061),
 ('baby_sitter', 15.035270575578268),
 ('baby_carriers', 4.1005283387940725),
 ('baby_arrives', 20.020856380012837),
 ('baby_groups', 3.4748180885780884),
 ('baby_needs', 2.4441079416662874),
 ('baby_shower', 2.744775561464593),
 ('baby_jars', 3.2197043563240317),
 ('baby_clinic', 6.103732659150479),
 ('baby_lead', 3.0668959880532785),
 ('baby_swing

In [16]:
_food = [item for item in bigram_transformer.export_phrases().items() if '_food' in item[0]]
_food

[('kosher_food', 4.1910697091931555),
 ('dog_food', 6.982126120364911),
 ('finger_food', 23.564871025461954),
 ('other_foods', 3.431096073696485),
 ('forbidden_foods', 33.389919435763645),
 ('certain_foods', 48.988031739693525),
 ('allergenic_foods', 31.45427193224111),
 ('multiple_food', 9.298935917272315),
 ('trigger_foods', 3.9341596918271358),
 ('jarred_foods', 19.73040693931488),
 ('new_foods', 5.721158548714751),
 ('particular_foods', 5.2993989781511655),
 ('plain_foods', 4.873957856871707),
 ('processed_food', 22.479670628670764),
 ('fast_food', 11.052859672049678),
 ('processed_foods', 77.07766449190298),
 ('proper_food', 5.923902578426851),
 ('frozen_food', 2.67475010654125),
 ('junk_food', 41.106705531387234),
 ('indian_food', 5.927608552842909),
 ('tinned_food', 4.037768044314114),
 ('fatty_foods', 57.96572970370147),
 ('certain_food', 2.398758156813495),
 ('these_foods', 4.20370176290699),
 ('range_of_foods', 4.769988490823377),
 ('bbc_food', 6.749523506028031),
 ('chinese_

In [17]:
formula = [item for item in bigram_transformer.export_phrases().items() if 'formula' in item[0]]
formula

[('add_formula', 5.487967164755832),
 ('formula_feed', 6.634845514894406),
 ('formula_milk', 6.402857002228837),
 ('based_formula', 8.19553442773525),
 ('formula_powder', 9.510120083975638),
 ('hypoallergenic_formula', 32.734828681451496),
 ('special_formulas', 2.849586487015529),
 ('soya_formula', 12.513803183142393),
 ('switched_to_formula', 18.675470351706416),
 ('introducing_formula', 11.935587569135327),
 ('onto_formula', 8.572533348055096),
 ('introduce_formula', 5.473672386273695),
 ('bottle_of_formula', 3.424693100627787),
 ('introduced_formula', 6.596658075844686),
 ('add_the_formula', 5.11937235518268),
 ('prescribed_formula', 2.219716206693347),
 ('comfort_formula', 2.9006581610097197),
 ('formula_feeds', 5.506564068413435),
 ('made_formula', 2.121590873898541),
 ('carton_of_formula', 8.585071383341111),
 ('powdered_formula', 44.84570183140028),
 ('formula_contains', 2.562611533222873),
 ('formula_fed', 21.95806352977246),
 ('sma_formula', 2.91305413605677),
 ('formulas_are'

In [18]:
bottle = [item for item in bigram_transformer.export_phrases().items() if 'bottle' in item[0]]
bottle

[('bottled_water', 96.62018433040697),
 ('use_bottled', 3.1517028659729225),
 ('bottled_waters', 36.6159529648494),
 ('bottle_feed', 6.526099174233689),
 ('bigger_bottles', 3.247733720528142),
 ('bottles_in_advance', 28.686418598044977),
 ('making_bottles', 4.639261881432918),
 ('mam_bottles', 115.01750942542313),
 ('tippee_bottles', 47.99154302615666),
 ('oz_bottles', 21.345981761004765),
 ('glass_bottles', 8.694545995691824),
 ('sterilised_bottle', 33.962666371154945),
 ('breast_or_bottle', 2.203804007747061),
 ('bottle_feeding', 17.154489706633573),
 ('dds_bottles', 2.312245203202101),
 ('bottle_of_formula', 3.424693100627787),
 ('bottle_feeds', 2.658098480273045),
 ('warm_bottle', 2.047676089238977),
 ('sterile_bottle', 17.424324486070798),
 ('bottle_filled', 2.7437888557941363),
 ('fill_bottles', 4.546827208739399),
 ('made_bottles', 2.49258508906151),
 ('water_bottles', 2.4451328585585435),
 ('pump_bottle', 3.100835236794311),
 ('water_bottle', 3.9281609070824284),
 ('sports_bott

In [19]:
milk = [item for item in bigram_transformer.export_phrases().items() if 'milk' in item[0]]
milk

[('breast_milk', 21.044993701053393),
 ('formula_milk', 6.402857002228837),
 ('hungrier_milk', 5.085573821198861),
 ('rice_milk', 3.1743375646954),
 ('cows_milk', 99.29266860748923),
 ('milk_proteins', 9.932761369529025),
 ('milk_allergy', 10.41756130016438),
 ('soya_milk', 24.620686621625033),
 ('milk_protein', 18.81788820655051),
 ('milk_protien', 28.45851629595293),
 ('oatly_milk', 20.537894277918475),
 ('milk_cartons', 3.550433851235907),
 ('coconut_milk', 23.83862728686966),
 ('soy_milk', 9.65266180813233),
 ('benefits_of_breastmilk', 6.072714663385078),
 ('expressed_milk', 41.58763638830844),
 ('amount_of_milk', 3.114579513992867),
 ('milk_powder', 8.277674413028024),
 ('fat_milk', 3.5452801467804314),
 ('hazelnut_milk', 30.066737118574345),
 ('almond_milk', 51.40375263237642),
 ('milk_substitute', 4.011307476155953),
 ('hemp_milk', 8.950902199667535),
 ('drink_milk', 2.1776462296656023),
 ('oat_milk', 29.96111626218591),
 ('milk_substitutes', 7.806801918507023),
 ('nut_milks', 1

In [20]:
bacteria = [item for item in bigram_transformer.export_phrases().items() if 'bacteria' in item[0]]
bacteria

[('contain_bacteria', 7.150618906108014),
 ('kill_the_bacteria', 142.16123178869967),
 ('bacteria_in_the_powder', 18.839905731200034),
 ('gut_bacteria', 31.602963375227663),
 ('beneficial_bacteria', 8.275843200977803),
 ('bacteria_in_the_gut', 18.589978456016272),
 ('bad_bacteria', 3.7848300346296404),
 ('germs_and_bacteria', 12.445351989256695),
 ('any_bacteria', 7.796038810614482),
 ('bacteria_present', 9.38939906624026),
 ('risk_of_bacteria', 4.879013452510164),
 ('bacteria_breeds', 17.72109902818073),
 ('friendly_bacteria', 52.157455550434925),
 ('good_bacteria', 3.821739232946373),
 ('bacteria_build', 2.969655939148683),
 ('ground_for_bacteria', 42.96024006831692),
 ('bacteria_in_the_milk', 2.076470878930939),
 ('bacterial_growth', 109.27843553472235),
 ('anti_bacterial', 512.1061601396458),
 ('kills_bacteria', 45.44504837888856),
 ('bacteria_viruses', 8.151705552963136),
 ('kill_bacteria', 83.49151708225219),
 ('viral_or_bacterial', 62.534830857347416),
 ('bacterial_wipes', 19.06

In [21]:
cereal = [item for item in bigram_transformer.export_phrases().items() if 'cereal' in item[0]]
cereal

[('cereal_or_toast', 4.943107922579185),
 ('cereal_bars', 144.7536850351884),
 ('chocolate_cereal', 2.6465203178562544),
 ('cereal_or_porridge', 2.240561336184306),
 ('cereal_bar', 112.79898760160306),
 ('rice_cereal', 3.259280606679975),
 ('cereal_toast', 4.943107922579185),
 ('gluten_cereals', 5.860542471819645),
 ('breakfast_cereal', 16.81930497632593),
 ('cereal_etc', 2.0733620109476893),
 ('adult_cereals', 2.816325691623138),
 ('bowl_of_cereal', 45.730175812637164),
 ('breakfast_cereals', 38.36919947824216),
 ('baby_cereal', 2.5499130726869557),
 ('fortified_cereals', 154.86795212765958),
 ('fibre_cereal', 5.210940552859303),
 ('sugary_cereals', 28.71319033530572),
 ('cereals_etc', 3.7190131163190943),
 ('fortified_cereal', 15.189337356206904),
 ('cereal_for_breakfast', 9.446458959306343),
 ('thickeners_and_cereals', 255.39627192982456),
 ('cereals_rice', 2.3736487037338985),
 ('baby_cereals', 4.072052447552448),
 ('toast_cereal', 3.4221516387086663),
 ('toast_or_cereal', 6.083825

In [22]:
porridge = [item for item in bigram_transformer.export_phrases().items() if 'porridge' in item[0]]
porridge

[('porridge_for_breakfast', 23.002903555061792),
 ('porridge_oats', 323.29487541267184),
 ('oats_porridge', 4.007787711727337),
 ('cereal_or_porridge', 2.240561336184306),
 ('baby_porridge', 8.080941815826687),
 ('porridge_mixed', 3.8574016812685255),
 ('rice_porridge', 3.1288582363852093),
 ('breakfast_porridge', 12.386178837340966),
 ('porridge_made', 2.509453309063986),
 ('banana_porridge', 11.602383510210267),
 ('adult_porridge', 2.386529420912506),
 ('proper_porridge', 6.765367213281093),
 ('porridge_or_weetabix', 14.979927840554637),
 ('baby_riceporridge', 22.5529058633674),
 ('creamy_porridge', 45.80623179007645),
 ('spoons_of_porridge', 14.71195059832647),
 ('bowl_of_porridge', 22.99536965661145),
 ('weetabix_porridge', 13.481935056499173),
 ('baby_porridges', 12.529392146315223),
 ('plain_porridge', 4.526629449837349),
 ('weetabix_or_porridge', 26.963870112998347),
 ('porridge_weetabix', 16.477920624610103),
 ('oatbran_porridge', 269.0087041647265),
 ('porridge_with_greek', 15

In [23]:
oats = [item for item in bigram_transformer.export_phrases().items() if 'oats' in item[0]]
oats

[('porridge_oats', 323.29487541267184),
 ('oats_in_the_bath', 19.01365136845476),
 ('g_oats', 2.830953861150156),
 ('rolled_oats', 212.83022660818713),
 ('oats_porridge', 4.007787711727337),
 ('goats_cheese', 91.09065209618306),
 ('overnight_oats', 79.72235913082595),
 ('their_throats', 18.785615376025213),
 ('goats_milk', 74.78265671103186),
 ('oats_in_a_muslin', 162.15636313004734),
 ('handful_of_oats', 62.418640797513135),
 ('use_goats', 2.495098102228564),
 ('sore_throats', 179.9667760396009),
 ('plain_oats', 2.249196582385069),
 ('pasta_oats', 3.7094592872886647),
 ('oats_beans', 4.671170954363504),
 ('oats_in_a_sock', 46.520268111079154),
 ('flour_oats', 2.3404011173408894),
 ('nannycare_goats', 473.41747967479677),
 ('allergic_to_goats', 12.265476566614007),
 ('winter_coats', 112.4189703516433),
 ('shoes_coats', 37.73296493628051),
 ('coats_and_bags', 6.750562253651751),
 ('bloats_me', 6.114310564788937),
 ('oatso_simple', 328.6137133182844),
 ('peoples_throats', 432.34035300867

In [24]:
fruit = [item for item in bigram_transformer.export_phrases().items() if 'fruit' in item[0]]
fruit

[('fruit_or_veg', 4.568350284283451),
 ('fresh_fruitveg', 41.33221122754255),
 ('fruit_and_veg', 42.60973629353268),
 ('fresh_fruit', 27.421034530101274),
 ('fruit_veg', 11.90027049362726),
 ('citrus_fruit', 82.23231938089589),
 ('kiwi_fruit', 30.251889757643454),
 ('fruits_and_veggies', 79.4040602538151),
 ('fruit_and_vegs', 6.05922353332917),
 ('veg_and_fruit', 4.004356422026235),
 ('dried_fruit', 33.1861782226617),
 ('fruit_juice', 26.553544828799904),
 ('tropical_fruit', 97.24171359799146),
 ('kiwi_fruits', 130.4467675994549),
 ('dried_fruits', 109.53748817183052),
 ('fruit_and_fibre', 5.462146761796735),
 ('followed_by_fruit', 2.291926818885613),
 ('fruit_salad', 12.663777184657969),
 ('piece_of_fruit', 26.12524518908947),
 ('fruit_and_vegetables', 22.671961501625535),
 ('fruit_loaf', 14.503968142772187),
 ('fruit_compote', 65.78585550471672),
 ('seeds_fruit', 2.195849400361798),
 ('other_fruits', 2.1799826574139596),
 ('fruits_and_veg', 12.038167714168923),
 ('portions_of_fruit',

In [25]:
veg = [item for item in bigram_transformer.export_phrases().items() if 'veg' in item[0]]
veg

[('fruit_or_veg', 4.568350284283451),
 ('fresh_fruitveg', 41.33221122754255),
 ('fruit_and_veg', 42.60973629353268),
 ('fruit_veg', 11.90027049362726),
 ('frozen_veg', 32.21285797835376),
 ('root_veg', 63.3929101177111),
 ('steamed_veg', 68.60878246916835),
 ('meat_and_veg', 5.455764840227622),
 ('vegetable_oils', 119.24507244150924),
 ('vegetable_oil', 104.80978254755718),
 ('hydrogenated_vegetable', 154.29345521992582),
 ('fruits_and_veggies', 79.4040602538151),
 ('fruit_and_vegs', 6.05922353332917),
 ('fresh_veg', 10.812688693451426),
 ('veg_and_fruit', 4.004356422026235),
 ('vegetable_protein', 5.16405051391882),
 ('veggie_stock', 5.196404867380031),
 ('ml_vegetable', 8.149689997340836),
 ('whatever_veg', 2.816027893821858),
 ('veg_pulses', 5.7824613958722955),
 ('ml_veg', 2.4434098118175585),
 ('veg_stock', 9.526954398315418),
 ('vegan_recipes', 10.656531157996355),
 ('vegan_chocolate', 2.2828663345298414),
 ('vegetable_stock', 97.35617354473764),
 ('veg_stir', 2.2640325042039677)

In [26]:
pure = [item for item in bigram_transformer.export_phrases().items() if 'pure' in item[0]]
#search pure rather than puree because the cleaning may have removed accented e.
pure

[('purely_down', 5.131735027021457),
 ('pure_cotton', 2.878482120200499),
 ('called_pure', 6.8383695113923215),
 ('pure_potions', 545.1955846510523),
 ('tomato_puree', 268.8577798838424),
 ('purely_because', 35.13199008925209),
 ('tom_puree', 257.74795426880763),
 ('pure_nuff', 300.00180319422975),
 ('fruit_puree', 36.346801068145105),
 ('pureed_food', 11.574688467528587),
 ('pureed_fruit', 37.31774623444222),
 ('fruit_pures', 19.942956196311762),
 ('pureed_veg', 50.62699588412017),
 ('pured_veg', 69.53409828536437),
 ('smooth_puree', 38.00950467618464),
 ('homemade_purees', 21.84667649629888),
 ('purees_and_finger', 18.573484381312863),
 ('veg_puree', 15.68069869719281),
 ('some_pureed', 2.4143819343786297),
 ('puree_mixed', 10.393781062952929),
 ('pureed_banana', 6.536277199246936),
 ('pureed_sweet', 4.658016024360512),
 ('veg_purees', 27.040379343447167),
 ('pure_aloe', 61.8324327194214),
 ('down_pure', 2.178135066753846),
 ('pure_fruit', 18.149575282213192),
 ('pure_juice', 77.1072

In [27]:
for_ = [item for item in bigram_transformer.export_phrases().items() if 'for_' in item[0]]
for_

[('microwave_for_secs', 73.96051990939691),
 ('cooled_for_mins', 11.13320056083105),
 ('thanks_for_replying', 61.16530814195419),
 ('thanks_for_the_advice', 3.7152177688987815),
 ('porridge_for_breakfast', 23.002903555061792),
 ('thanks_for_your', 2.322495064111071),
 ('worked_for_us', 5.109454913587071),
 ('ready_for_bed', 5.692427619794773),
 ('thanks_for_sharing', 22.959729042916653),
 ('better_for_the_environment', 3.9592707587925853),
 ('thanks_for_the_info', 6.003140012481455),
 ('bleach_for_the_loo', 10.078378261433949),
 ('worked_for_me', 2.4428753916396104),
 ('sorry_for_the_delay', 6.303843706853335),
 ('responsibility_for_their', 2.215961735597929),
 ('thanks_for_posting', 14.646447263061706),
 ('thanks_for_the_reply', 10.864626273152625),
 ('thanks_for_coming', 2.5654609833184074),
 ('thanks_for_mentioning', 5.398638991172294),
 ('works_for_us', 3.437944094629979),
 ('value_for_money', 36.885201402801194),
 ('thanks_for_reading', 4.35374112191314),
 ('leave_for_mins', 2.639

In [28]:
_for = [item for item in bigram_transformer.export_phrases().items() if item[0].endswith('_for')]
_for

[]

In [29]:
fish = [item for item in bigram_transformer.export_phrases().items() if 'fish' in item[0]]
fish

[('fish_and_chips', 46.507425305253136),
 ('fish_cakes', 22.21556346181999),
 ('white_fish', 7.10339454620524),
 ('oily_fish', 354.73128655982833),
 ('fish_fingers', 74.73197367781917),
 ('tuna_fish', 2.1069897871493573),
 ('raw_fish', 9.393254469686786),
 ('incredibly_selfish', 9.600730396359559),
 ('nuggets_fish', 7.5699319822643165),
 ('eat_fish', 3.6077317162204703),
 ('meat_and_fish', 5.590830486207697),
 ('fish_pie', 80.67976972653067),
 ('thai_fish', 19.799506970418225),
 ('fish_curry', 7.514971505810912),
 ('fish_paste', 5.49537336321812),
 ('steamed_fish', 8.521306797395185),
 ('meat_or_fish', 6.142039125692962),
 ('salmon_fish', 2.569401667916869),
 ('fish_oil', 12.74529364360829),
 ('homemade_fish', 4.9359176993372),
 ('pie_fish', 5.110935638784747),
 ('fisher_price', 764.7276373959778),
 ('chicken_or_fish', 2.183082431290217),
 ('omega_fish', 33.5393783555839),
 ('fish_oils', 72.06400187144388),
 ('fish_eggs', 2.3073975561070084),
 ('raw_shellfish', 276.99426532769553),
 ('

In [30]:
model.wv.most_similar('baby_food', topn=70)

[('jars', 0.6632078886032104),
 ('ready_made', 0.6512479186058044),
 ('cartons', 0.6297696828842163),
 ('child_friendly', 0.5986317992210388),
 ('ingredients', 0.592647910118103),
 ('pouches', 0.5801127552986145),
 ('sauces', 0.5756595134735107),
 ('ready_meals', 0.5654070973396301),
 ('brands', 0.5648307800292969),
 ('packets', 0.561143159866333),
 ('what_brand', 0.5608521699905396),
 ('most_often', 0.5594455003738403),
 ('powdered_milk', 0.5592845678329468),
 ('tins', 0.5544024109840393),
 ('pre_made', 0.5535987019538879),
 ('organic_food', 0.5520203113555908),
 ('brand', 0.5480930805206299),
 ('value', 0.5461275577545166),
 ('tubs', 0.5454987287521362),
 ('little_pots', 0.5453415513038635),
 ('jar_food', 0.5437083840370178),
 ('currently_use', 0.5428208112716675),
 ('baby_jars', 0.5425620675086975),
 ('wipes_are', 0.5418193340301514),
 ('frozen_food', 0.5409688949584961),
 ('batches', 0.5404052138328552),
 ('own_brand', 0.5389884114265442),
 ('aldi_mamia', 0.5383155941963196),
 ('bi

In [31]:
model.wv.most_similar('pureed', topn=70)
#there are specific fruits and veggies in here.
#TODO: make a list of all english fruits and veggies
#sweet_corn
#steamed (x)
#peach


[('mashed_up', 0.8917757868766785),
 ('mashed', 0.883613109588623),
 ('blended', 0.8622050285339355),
 ('pured', 0.8479769825935364),
 ('puree', 0.826306164264679),
 ('potato', 0.8251627087593079),
 ('sweet_potato', 0.8197069764137268),
 ('mashed_potato', 0.8170260190963745),
 ('steamed', 0.816729724407196),
 ('chopped_up', 0.8154729604721069),
 ('brown_rice', 0.8104413151741028),
 ('tinned', 0.8096159100532532),
 ('mushed_up', 0.8044378161430359),
 ('butternut_squash', 0.7989594340324402),
 ('mashed_potatoes', 0.7988768815994263),
 ('sweetcorn', 0.7983130812644958),
 ('pureed_fruit', 0.7972396612167358),
 ('mashed_banana', 0.7964199185371399),
 ('cut_up', 0.795372724533081),
 ('cheese_spread', 0.7952924966812134),
 ('tofu', 0.7952902913093567),
 ('cauliflower', 0.7945024371147156),
 ('avocado', 0.7942897081375122),
 ('spinach', 0.7937605381011963),
 ('green_beans', 0.7934799194335938),
 ('parsnip', 0.7917948961257935),
 ('bread_sticks', 0.7914296388626099),
 ('broccoli', 0.79135298728

In [32]:
model.wv.most_similar('puree', topn=70)

[('pureed', 0.8263062238693237),
 ('mashed', 0.8188937306404114),
 ('fruit_puree', 0.7937485575675964),
 ('baby_porridge', 0.7910634279251099),
 ('purees', 0.7902653217315674),
 ('finger_food', 0.785697340965271),
 ('rice', 0.7757613062858582),
 ('potato', 0.7723339796066284),
 ('porridge', 0.7720587253570557),
 ('mashed_banana', 0.7702802419662476),
 ('pureed_veg', 0.7680514454841614),
 ('mash', 0.7676036357879639),
 ('baby_rice', 0.7672333717346191),
 ('pured', 0.7668317556381226),
 ('mashed_up', 0.7653437256813049),
 ('yogurt', 0.7637194991111755),
 ('pureed_fruit', 0.7631524801254272),
 ('pures', 0.7627577185630798),
 ('yoghurt', 0.7503347992897034),
 ('blended', 0.7476862668991089),
 ('blend', 0.746124804019928),
 ('sweet_potato', 0.738776445388794),
 ('mashed_potato', 0.7336557507514954),
 ('gravy', 0.7318558692932129),
 ('rusk', 0.7286402583122253),
 ('carrots', 0.7277993559837341),
 ('fruit_purees', 0.7264474034309387),
 ('finger_foods', 0.7242370247840881),
 ('vegetables', 0.7

In [33]:
model.wv.most_similar('pure', topn=100)

[('powdered', 0.7367174029350281),
 ('margarine', 0.7296808362007141),
 ('flavoured', 0.7282230257987976),
 ('sweetened', 0.7261917591094971),
 ('oat_milk', 0.7253637313842773),
 ('vegetable_oil', 0.7241896390914917),
 ('almond_milk', 0.7209106683731079),
 ('coconut_milk', 0.7202200293540955),
 ('diluted', 0.7200751900672913),
 ('oat', 0.7149084806442261),
 ('yeast', 0.7146998047828674),
 ('syrup', 0.7140776515007019),
 ('butter', 0.7091851234436035),
 ('full_fat', 0.7076459527015686),
 ('soy', 0.7049808502197266),
 ('marg', 0.7046194076538086),
 ('thickened', 0.7042940855026245),
 ('oats', 0.7027295231819153),
 ('whey', 0.7020899057388306),
 ('ordinary', 0.6995634436607361),
 ('wholegrain', 0.6980748176574707),
 ('plain', 0.69725102186203),
 ('starch', 0.6961092948913574),
 ('low_calorie', 0.6934932470321655),
 ('fat_free', 0.6930691003799438),
 ('puree', 0.6876533627510071),
 ('whole_milk', 0.6875491142272949),
 ('soya_milk', 0.6874445676803589),
 ('fruit_purees', 0.6871779561042786)

In [34]:
model.wv.most_similar('porridge', topn=70)

[('weetabix', 0.9047213196754456),
 ('baby_porridge', 0.8838602900505066),
 ('yoghurt', 0.8715159296989441),
 ('fruit_puree', 0.8605647683143616),
 ('cereal', 0.8562952280044556),
 ('yogurt', 0.8484867811203003),
 ('mashed_banana', 0.8305881023406982),
 ('whole_milk', 0.8207049369812012),
 ('rusk', 0.8152687549591064),
 ('baby_rice', 0.8104282021522522),
 ('ready_brek', 0.8078296184539795),
 ('skimmed_milk', 0.807253897190094),
 ('banana', 0.8064581751823425),
 ('fromage_frais', 0.8062134981155396),
 ('readybrek', 0.7851037979125977),
 ('greek_yoghurt', 0.7812036871910095),
 ('almond_milk', 0.7773952484130859),
 ('oat_milk', 0.7763589024543762),
 ('soya_milk', 0.7752267718315125),
 ('rice_pudding', 0.7721554040908813),
 ('puree', 0.7720587253570557),
 ('orange_juice', 0.7695547342300415),
 ('yog', 0.769315779209137),
 ('custard', 0.7692141532897949),
 ('fruit_purees', 0.7687221169471741),
 ('fruit_pure', 0.7684205174446106),
 ('blueberries', 0.7674225568771362),
 ('apple_juice', 0.7667

In [35]:
model.wv.most_similar('cereal', topn=70)

[('porridge', 0.8562952280044556),
 ('fruit', 0.8440166711807251),
 ('yogurt', 0.8349622488021851),
 ('weetabix', 0.8340936303138733),
 ('fromage_frais', 0.8275046944618225),
 ('cereals', 0.8251352906227112),
 ('yoghurt', 0.8162539601325989),
 ('yogurts', 0.8067172169685364),
 ('skimmed_milk', 0.8012534976005554),
 ('yoghurts', 0.8006985783576965),
 ('peanut_butter', 0.7911021709442139),
 ('breakfast_cereal', 0.7903540134429932),
 ('fruit_juice', 0.7884752750396729),
 ('muesli', 0.7743737101554871),
 ('milkshake', 0.7739576697349548),
 ('cornflakes', 0.7691288590431213),
 ('bread', 0.7655858993530273),
 ('shreddies', 0.7645879983901978),
 ('yog', 0.7633464932441711),
 ('biscuits', 0.759250819683075),
 ('baby_porridge', 0.7590208053588867),
 ('cheerios', 0.7588184475898743),
 ('fruitveg', 0.7531624436378479),
 ('desserts', 0.7500132322311401),
 ('dessert', 0.7499061822891235),
 ('almond_milk', 0.7491176724433899),
 ('ready_brek', 0.7463222146034241),
 ('cheese', 0.7451317310333252),
 ('

In [36]:
model.wv.most_similar('sterilised', topn=70)

[('sealed', 0.7275084853172302),
 ('washed', 0.7258801460266113),
 ('boiled', 0.7029150128364563),
 ('sterilized', 0.6967759728431702),
 ('sterile', 0.6908904910087585),
 ('dissolved', 0.6747462153434753),
 ('rinsed', 0.6737047433853149),
 ('reheated', 0.6709648966789246),
 ('kept_in_the_fridge', 0.6644665598869324),
 ('cooled', 0.662738561630249),
 ('soaked', 0.6590367555618286),
 ('binned', 0.6586465835571289),
 ('warmed', 0.6546679735183716),
 ('heated_up', 0.6537561416625977),
 ('refrigerated', 0.6432721018791199),
 ('microwaved', 0.6399952173233032),
 ('defrosted', 0.6396204829216003),
 ('thrown_away', 0.6376843452453613),
 ('stored', 0.6365488767623901),
 ('sterilising', 0.6301044225692749),
 ('cooled_down', 0.6198455691337585),
 ('heated', 0.6189181208610535),
 ('packed', 0.6169779300689697),
 ('piping_hot', 0.6160373091697693),
 ('sterilise', 0.6137729287147522),
 ('discarded', 0.6118220686912537),
 ('frozen', 0.6088703870773315),
 ('bottled', 0.6070979237556458),
 ('steralised

In [37]:
model.wv.most_similar('isnt_sterile', topn=70)

[('kills_any', 0.8579500913619995),
 ('bacteria_in_the_formula', 0.8087029457092285),
 ('powder_itself', 0.8007192611694336),
 ('bacteria_in_the_powder', 0.8001772165298462),
 ('added_to_boiling', 0.7965834736824036),
 ('kills_the_bacteria', 0.7854009866714478),
 ('above_degrees', 0.7714803218841553),
 ('contains_sugar', 0.7581871747970581),
 ('doesnt_kill', 0.7544713616371155),
 ('also_contains', 0.7531009316444397),
 ('bacteria_in_the_milk', 0.7510657906532288),
 ('teaspoon_baking', 0.7507692575454712),
 ('natural_mineral', 0.750613808631897),
 ('cocunut', 0.7466084957122803),
 ('not_sterile', 0.7456137537956238),
 ('modified_starch', 0.7374883890151978),
 ('breaks_down', 0.7373273968696594),
 ('cooler_than', 0.734498143196106),
 ('pasturisation', 0.7341873645782471),
 ('neutralises', 0.7327064871788025),
 ('naturally_occuring', 0.7322784662246704),
 ('water_to_kill', 0.7322733402252197),
 ('mixed_with_water', 0.7320549488067627),
 ('contains_lactose', 0.7291804552078247),
 ('just_ad

In [38]:
model.wv.most_similar('bacteria', topn=70)

[('any_bacteria', 0.8274815678596497),
 ('spores', 0.8252037167549133),
 ('toxins', 0.7565704584121704),
 ('cells', 0.7511088252067566),
 ('saliva', 0.7414253354072571),
 ('harmful_bacteria', 0.7360973358154297),
 ('fluoride', 0.7261933088302612),
 ('chemicals', 0.7238971590995789),
 ('itself', 0.7234700918197632),
 ('acid', 0.7232255339622498),
 ('formula_powder', 0.7187797427177429),
 ('antibodies', 0.7139322757720947),
 ('foreskin', 0.709734320640564),
 ('chlorine', 0.7085893750190735),
 ('germs', 0.7085462212562561),
 ('enzymes', 0.7076816558837891),
 ('production', 0.697669506072998),
 ('good_bacteria', 0.6971322298049927),
 ('sodium', 0.6963452100753784),
 ('moisture', 0.6957602500915527),
 ('gut', 0.6907584071159363),
 ('proteins', 0.6852127909660339),
 ('multiply', 0.6828136444091797),
 ('kill_off', 0.6809341907501221),
 ('listeria', 0.6809327602386475),
 ('nutrients', 0.6793967485427856),
 ('substance', 0.6764995455741882),
 ('hormone', 0.6762189269065857),
 ('glans', 0.675804

In [39]:
model.wv.most_similar('contaminated', topn=70)

[('broken_down', 0.7578365206718445),
 ('listeria', 0.7347068786621094),
 ('ingested', 0.7226502299308777),
 ('pasteurised', 0.720508337020874),
 ('salmonella', 0.7187277674674988),
 ('toxic', 0.7146514654159546),
 ('safe_to_eat', 0.7137652039527893),
 ('digested', 0.7082943916320801),
 ('killed_off', 0.7082933783531189),
 ('illegal', 0.7028290629386902),
 ('absorbed', 0.7007914781570435),
 ('produced', 0.7006914019584656),
 ('derived', 0.6998346447944641),
 ('slaughtered', 0.6981883645057678),
 ('lethal', 0.6970028877258301),
 ('harmful', 0.6929693818092346),
 ('protected', 0.6847737431526184),
 ('prevented', 0.677053689956665),
 ('dangerous', 0.6761312484741211),
 ('labelled', 0.6741476655006409),
 ('consumed', 0.6741132736206055),
 ('formed', 0.6740157008171082),
 ('processed', 0.669874370098114),
 ('consuming', 0.6679480075836182),
 ('saturated', 0.6671556234359741),
 ('obtained', 0.6659201979637146),
 ('very_dangerous', 0.6657426953315735),
 ('banned', 0.663032591342926),
 ('best_

In [40]:
model.wv.most_similar('contaminated', topn=70)

[('broken_down', 0.7578365206718445),
 ('listeria', 0.7347068786621094),
 ('ingested', 0.7226502299308777),
 ('pasteurised', 0.720508337020874),
 ('salmonella', 0.7187277674674988),
 ('toxic', 0.7146514654159546),
 ('safe_to_eat', 0.7137652039527893),
 ('digested', 0.7082943916320801),
 ('killed_off', 0.7082933783531189),
 ('illegal', 0.7028290629386902),
 ('absorbed', 0.7007914781570435),
 ('produced', 0.7006914019584656),
 ('derived', 0.6998346447944641),
 ('slaughtered', 0.6981883645057678),
 ('lethal', 0.6970028877258301),
 ('harmful', 0.6929693818092346),
 ('protected', 0.6847737431526184),
 ('prevented', 0.677053689956665),
 ('dangerous', 0.6761312484741211),
 ('labelled', 0.6741476655006409),
 ('consumed', 0.6741132736206055),
 ('formed', 0.6740157008171082),
 ('processed', 0.669874370098114),
 ('consuming', 0.6679480075836182),
 ('saturated', 0.6671556234359741),
 ('obtained', 0.6659201979637146),
 ('very_dangerous', 0.6657426953315735),
 ('banned', 0.663032591342926),
 ('best_

## We can find words that are close to a set of words as well.

In [60]:
model.wv.most_similar(['very_dangerous','formula'], topn=10)

[('not_sterile', 0.7310904860496521),
 ('formula_milk', 0.6959844827651978),
 ('unsafe', 0.6930491924285889),
 ('harmful', 0.6888231635093689),
 ('dangerous', 0.6882530450820923),
 ('not_recommended', 0.6762654781341553),
 ('breastmilk', 0.6700856685638428),
 ('breast_milk', 0.6665492057800293),
 ('sterile', 0.6651689410209656),
 ('bpa', 0.6646931767463684)]

In [64]:
model.wv.most_similar(['formula_contains'], topn=40)

[('vitamins_minerals', 0.8106856942176819),
 ('hydrolysate', 0.7930810451507568),
 ('caseinate', 0.7891420125961304),
 ('yeasts', 0.7845163941383362),
 ('whey', 0.7800481915473938),
 ('propionate', 0.7796724438667297),
 ('colourants', 0.776107907295227),
 ('non_haem', 0.7740671634674072),
 ('fatty_acids', 0.7704779505729675),
 ('foods_containing', 0.7699160575866699),
 ('absorbable', 0.7692038416862488),
 ('trans_fats', 0.7654028534889221),
 ('formulas_are', 0.765007734298706),
 ('phosphate', 0.7583925127983093),
 ('cancer_causing', 0.7583493590354919),
 ('citrate', 0.7577664256095886),
 ('thimerosal', 0.7568718791007996),
 ('also_contains', 0.7562380433082581),
 ('bisulphite', 0.7550860047340393),
 ('decreases', 0.7528793811798096),
 ('carbohydrates', 0.7520921230316162),
 ('casein', 0.7514002323150635),
 ('dairy_gluten', 0.7504690885543823),
 ('intakes', 0.7499444484710693),
 ('gmo', 0.7491322159767151),
 ('colours_flavours', 0.7489995360374451),
 ('naturally_occuring', 0.74895590543

In [63]:
model.wv.most_similar(['bpa'], topn=40)

[('arsenic', 0.8115674257278442),
 ('oestrogen', 0.7964176535606384),
 ('mercury', 0.7922999262809753),
 ('sodium', 0.7916605472564697),
 ('fluoride', 0.7734161019325256),
 ('parabens', 0.7723528742790222),
 ('aluminium', 0.7576263546943665),
 ('toxicity', 0.7467406988143921),
 ('plastics', 0.7460504770278931),
 ('tannin', 0.7447837591171265),
 ('fatty_acids', 0.739966869354248),
 ('chemicals', 0.7391164898872375),
 ('amino_acids', 0.7389072179794312),
 ('potassium', 0.7369123697280884),
 ('aspartame', 0.7349771857261658),
 ('chemical', 0.7339134812355042),
 ('preservative', 0.7317711114883423),
 ('cancer_causing', 0.7314833998680115),
 ('refined_sugar', 0.7294902801513672),
 ('additives', 0.7273610234260559),
 ('pesticides', 0.726372480392456),
 ('minerals', 0.7217450141906738),
 ('molecules', 0.7212138772010803),
 ('cosmetic', 0.7207120656967163),
 ('infant_formula', 0.7197909355163574),
 ('naturally_occurring', 0.7191856503486633),
 ('aluminum', 0.7182139754295349),
 ('nasties', 0.7

In [41]:
model.wv.most_similar(['veg','puree'], topn=10)

[('vegetables', 0.8697404265403748),
 ('rice', 0.8630017638206482),
 ('veggies', 0.8568155169487),
 ('pasta', 0.8370674252510071),
 ('potato', 0.8305625915527344),
 ('mashed', 0.8222828507423401),
 ('pureed', 0.8215149641036987),
 ('gravy', 0.8214550614356995),
 ('mash', 0.8172215223312378),
 ('carrots', 0.8160039186477661)]

In [42]:
model.wv.most_similar(['veg','sterilised'], topn=20)

[('boiled', 0.800743818283081),
 ('frozen', 0.7704490423202515),
 ('veggies', 0.7600007653236389),
 ('fresh_veg', 0.7540833950042725),
 ('rice', 0.7523230910301208),
 ('potatoes', 0.7522738575935364),
 ('blended', 0.7455379366874695),
 ('frozen_veg', 0.7402791380882263),
 ('steamed', 0.733289897441864),
 ('defrosted', 0.7300256490707397),
 ('mashed', 0.726456344127655),
 ('cooked', 0.7260514497756958),
 ('vegetables', 0.7238166928291321),
 ('carrots', 0.7193670868873596),
 ('lentils', 0.7164996862411499),
 ('pureed', 0.7155889868736267),
 ('tinned', 0.7152246236801147),
 ('gravy', 0.7097538113594055),
 ('brown_rice', 0.7059224247932434),
 ('reheated', 0.7039092779159546)]

In [43]:
model.wv.most_similar(['veg','baby_food', 'veggie','vegetable','vegetables'], topn=20)

[('veggies', 0.8707243800163269),
 ('sauces', 0.8702512383460999),
 ('salads', 0.8631791472434998),
 ('lentils', 0.8581139445304871),
 ('soups', 0.8489151000976562),
 ('pies', 0.8480373024940491),
 ('frozen_veg', 0.8464733362197876),
 ('pulses', 0.8453024625778198),
 ('meats', 0.8444339632987976),
 ('quorn', 0.844214677810669),
 ('brown_rice', 0.8440425395965576),
 ('pasta_sauces', 0.8420789837837219),
 ('tinned', 0.8352919220924377),
 ('curries', 0.8352091908454895),
 ('corn', 0.8309424519538879),
 ('pasta', 0.8300403952598572),
 ('potatoes', 0.8296052813529968),
 ('ham', 0.8291753530502319),
 ('spinach', 0.8288935422897339),
 ('pasta_sauce', 0.8286823630332947)]

In [44]:
model.wv.most_similar(['home_made','homemade','puree'], topn=20)

[('hm', 0.8659756183624268),
 ('mashed_up', 0.8603705763816833),
 ('butternut_squash', 0.853517472743988),
 ('mashed', 0.8509690761566162),
 ('pureed', 0.8506379723548889),
 ('quiche', 0.8462623357772827),
 ('rice_pudding', 0.8435064554214478),
 ('sweet_potato', 0.840966522693634),
 ('garlic_bread', 0.8368018269538879),
 ('cottage_pie', 0.8361107707023621),
 ('custard', 0.8358594179153442),
 ('shepherds_pie', 0.8352732062339783),
 ('soups', 0.8348098993301392),
 ('soup', 0.8343158960342407),
 ('fish_pie', 0.8332914113998413),
 ('lasagne', 0.8316987156867981),
 ('potato', 0.8313376307487488),
 ('mashed_potato', 0.8290320634841919),
 ('gravy', 0.8286134004592896),
 ('spag_bol', 0.826851487159729)]

In [45]:
model.wv.most_similar(['home_made','homemade'], topn=20)

[('hm', 0.8983438014984131),
 ('cottage_pie', 0.8562551140785217),
 ('quiche', 0.8559617400169373),
 ('fish_pie', 0.855707585811615),
 ('lasagne', 0.8530722856521606),
 ('pies', 0.8459154367446899),
 ('stews', 0.8452112078666687),
 ('shepherds_pie', 0.842523992061615),
 ('pasta_bake', 0.8368378281593323),
 ('waffles', 0.836669921875),
 ('chicken_curry', 0.8343865275382996),
 ('spag_bol', 0.8341925740242004),
 ('garlic_bread', 0.8334532380104065),
 ('muffins', 0.8317611217498779),
 ('chocolate_cake', 0.8287607431411743),
 ('cheesy', 0.8283963799476624),
 ('bolognaise', 0.8269147276878357),
 ('meatballs', 0.8266160488128662),
 ('rice_pudding', 0.8264386057853699),
 ('pie', 0.8262291550636292)]

In [46]:
model.wv.most_similar(['pure','puree','baby','food','baby_food'], topn=20)

[('purees', 0.7827861905097961),
 ('proper_food', 0.7300967574119568),
 ('solid_food', 0.7121331095695496),
 ('dry_food', 0.7011641263961792),
 ('finger_food', 0.6986075043678284),
 ('pures', 0.6954949498176575),
 ('cereal', 0.6909707188606262),
 ('finger_foods', 0.68131422996521),
 ('rusks', 0.6810542345046997),
 ('fruit_purees', 0.6787837147712708),
 ('formula_milk', 0.6787124276161194),
 ('real_food', 0.6738409399986267),
 ('jars', 0.673393964767456),
 ('organic_food', 0.6718686819076538),
 ('soya_milk', 0.6709588766098022),
 ('rice', 0.6693932414054871),
 ('homemade_food', 0.666064977645874),
 ('fruitveg', 0.6649192571640015),
 ('vegetables', 0.6629579663276672),
 ('breastmilk', 0.6613625884056091)]

In [47]:
model.wv.most_similar(['pure','puree','baby','food'], topn=20)

[('purees', 0.7785037159919739),
 ('finger_food', 0.736261785030365),
 ('solid_food', 0.7361069917678833),
 ('proper_food', 0.7328143119812012),
 ('dry_food', 0.7062361240386963),
 ('finger_foods', 0.7053252458572388),
 ('cereal', 0.7019439935684204),
 ('pures', 0.6785394549369812),
 ('porridge', 0.677670955657959),
 ('wet_food', 0.675980806350708),
 ('soya_milk', 0.6758304834365845),
 ('yogurt', 0.6748199462890625),
 ('rice', 0.6738066673278809),
 ('lumpy_food', 0.6720040440559387),
 ('fruit_purees', 0.6701151728630066),
 ('baby_rice', 0.6697560548782349),
 ('rusk', 0.6676920056343079),
 ('spoon', 0.6674065589904785),
 ('real_food', 0.6659228801727295),
 ('pureed_food', 0.6650039553642273)]

In [48]:
model.wv.most_similar(positive = ['pure','puree','baby','food','weaning'], negative=['spirits', 'tea'], topn=20)

[('purees', 0.7228007316589355),
 ('solid_food', 0.6688770055770874),
 ('baby_rice', 0.6395720839500427),
 ('formula_milk', 0.6345375776290894),
 ('solids', 0.6342297792434692),
 ('finger_foods', 0.6248621344566345),
 ('formula', 0.6073945164680481),
 ('follow_on_milk', 0.6017842292785645),
 ('goats_milk', 0.5998283624649048),
 ('breastmilk', 0.5951478481292725),
 ('aptamil', 0.5884299874305725),
 ('blw', 0.5861135721206665),
 ('breast_milk', 0.5829591751098633),
 ('lumpy_food', 0.5822325944900513),
 ('textures', 0.5705565810203552),
 ('solid_foods', 0.5658881664276123),
 ('pures', 0.5656273365020752),
 ('rusks', 0.5624608397483826),
 ('milks', 0.5594924688339233),
 ('spoon_feeding', 0.5587582588195801)]

In [49]:
model.wv.most_similar(positive = ['baby_food','food','weaning'], negative=['spirits', 'tea'], topn=20)

[('blw', 0.5134778022766113),
 ('formula', 0.5068009495735168),
 ('foods', 0.49086955189704895),
 ('formula_milk', 0.4876461327075958),
 ('formulas', 0.4506283402442932),
 ('weaning_at_months', 0.44744157791137695),
 ('purees', 0.4449317157268524),
 ('follow_on_milk', 0.4423399567604065),
 ('product', 0.4407045841217041),
 ('weaning_before', 0.43600890040397644),
 ('products', 0.4339844286441803),
 ('wean', 0.4283212423324585),
 ('aptamil', 0.42795807123184204),
 ('solids', 0.42408454418182373),
 ('solid_food', 0.4203464686870575),
 ('baby_led', 0.4181869924068451),
 ('guidelines', 0.4139408469200134),
 ('milks', 0.40962058305740356),
 ('bottle_feeding', 0.406314879655838),
 ('solid_foods', 0.4017700254917145)]

In [50]:
model.wv.most_similar(['preservatives'], topn=20)

[('additives', 0.9097955226898193),
 ('flavourings', 0.8923246264457703),
 ('sweeteners', 0.8765747547149658),
 ('e_numbers', 0.8615750670433044),
 ('artificial_sweeteners', 0.8562761545181274),
 ('colourings', 0.8516984581947327),
 ('sweetners', 0.8473184704780579),
 ('minerals', 0.840736448764801),
 ('sodium', 0.8223258852958679),
 ('natural_sugar', 0.8209199905395508),
 ('grains', 0.8179807662963867),
 ('natural_sugars', 0.8175586462020874),
 ('parabens', 0.8161562085151672),
 ('nasties', 0.814775824546814),
 ('sugars', 0.8079137802124023),
 ('they_contain', 0.8018766641616821),
 ('chemicals', 0.8018603324890137),
 ('salts', 0.7987815141677856),
 ('fructose', 0.7977546453475952),
 ('carbohydrates', 0.7977396249771118)]

In [51]:
model.wv.most_similar(['what_brand'], topn=50)

[('most_often', 0.9138657450675964),
 ('currently_use', 0.8333656191825867),
 ('snacks_to_aldi', 0.8111298084259033),
 ('mamia_and_why', 0.7983057498931885),
 ('child_friendly', 0.7907569408416748),
 ('changing_from_your', 0.7817025780677795),
 ('current_brand', 0.7795124053955078),
 ('aldimamiaand_why', 0.761532723903656),
 ('wider_range', 0.7466215491294861),
 ('your_perception', 0.7452471852302551),
 ('whole_range', 0.739489734172821),
 ('currently_buy', 0.7350278496742249),
 ('cheaper_cuts', 0.7342355251312256),
 ('did_the_quality', 0.7330722212791443),
 ('think_w', 0.7296954393386841),
 ('organics_for_kids', 0.7267701029777527),
 ('set_reflect', 0.7241159677505493),
 ('cheap_version', 0.7232818007469177),
 ('these_types', 0.7224540710449219),
 ('online_surveys', 0.7175719141960144),
 ('sudeocreme', 0.717206597328186),
 ('wider_variety', 0.7165272235870361),
 ('products_from_the_aldi', 0.7163680791854858),
 ('three_packs', 0.7162688970565796),
 ('autoantibodies', 0.7162376046180725

In [52]:
model.wv.most_similar(['poisoning'], topn=50)

[('neglect', 0.7360648512840271),
 ('damaging', 0.7203733325004578),
 ('contracting', 0.7078014016151428),
 ('sexual', 0.6904181241989136),
 ('abusing', 0.687574565410614),
 ('ingesting', 0.6844332814216614),
 ('life_threatening', 0.682854175567627),
 ('deficiency', 0.6734163761138916),
 ('protecting', 0.6721208691596985),
 ('heart_disease', 0.6689537763595581),
 ('harming', 0.6671976447105408),
 ('sensitivity', 0.6637802124023438),
 ('overfeeding', 0.6633099317550659),
 ('intelligence', 0.6624796986579895),
 ('abuse', 0.6604698300361633),
 ('consuming', 0.6599719524383545),
 ('immune', 0.6577962040901184),
 ('very_serious', 0.6577288508415222),
 ('white_blood', 0.6562772989273071),
 ('toxoplasmosis', 0.6550978422164917),
 ('listeria', 0.6539000868797302),
 ('exposing', 0.6534973382949829),
 ('fatal', 0.6533563137054443),
 ('tooth_decay', 0.6497007012367249),
 ('presumably', 0.6491307616233826),
 ('subjecting', 0.6475749015808105),
 ('inherited', 0.6475026607513428),
 ('andyoung', 0.64

In [53]:
model.wv.most_similar(['address'], topn=50)

[('report', 0.7330910563468933),
 ('email', 0.7146645784378052),
 ('request', 0.7051324248313904),
 ('email_address', 0.6850564479827881),
 ('review', 0.6712112426757812),
 ('petition', 0.6705275177955627),
 ('e_mail', 0.6618393659591675),
 ('view', 0.6605817675590515),
 ('solicitor', 0.6506202816963196),
 ('upload', 0.6451637744903564),
 ('details', 0.6398294568061829),
 ('copy', 0.6339782476425171),
 ('claim', 0.6321592926979065),
 ('emails', 0.6320019364356995),
 ('print_out', 0.6254281997680664),
 ('agency', 0.6198511719703674),
 ('findings', 0.6188137531280518),
 ('status', 0.618386447429657),
 ('assess', 0.6174840927124023),
 ('reference', 0.6165534853935242),
 ('full_name', 0.6144192814826965),
 ('environmental_health', 0.6142573356628418),
 ('link', 0.6139795184135437),
 ('register', 0.611558198928833),
 ('prove', 0.6107545495033264),
 ('document', 0.6056265234947205),
 ('mp', 0.5999156832695007),
 ('facebook_page', 0.5929974913597107),
 ('quote', 0.5911537408828735),
 ('assess

In [54]:
#this one is strange, it returns a bunch of 
#names and email addresses and stuff
model.wv.most_similar(['some_info'], topn=50)

[('some_information', 0.8895822167396545),
 ('more_info', 0.8492863774299622),
 ('any_info', 0.8220314383506775),
 ('more_information', 0.7901368141174316),
 ('this_website', 0.7488385438919067),
 ('this_link', 0.7425857782363892),
 ('lots_of_information', 0.7407522201538086),
 ('this_page', 0.7383037805557251),
 ('this_forum', 0.7290728688240051),
 ('further_information', 0.7278512120246887),
 ('my_blog', 0.723211407661438),
 ('some_advice', 0.7229478359222412),
 ('facebook', 0.720672070980072),
 ('loads_of_info', 0.7175654172897339),
 ('facebook_group', 0.7160988450050354),
 ('updates', 0.711809515953064),
 ('lots_of_info', 0.7117425799369812),
 ('mse', 0.7115534543991089),
 ('info', 0.709985613822937),
 ('their_website', 0.7087144255638123),
 ('this_information', 0.7072226405143738),
 ('face_book', 0.7042806148529053),
 ('any_questions', 0.7024156451225281),
 ('topcashback', 0.6991915702819824),
 ('link', 0.6990742683410645),
 ('pdf', 0.6936559081077576),
 ('any_input', 0.6935021877

In [58]:
model.wv.most_similar(['baby_rice'], topn=70)

[('baby_porridge', 0.8808053731918335),
 ('porridge', 0.8104282021522522),
 ('purees', 0.7915183305740356),
 ('puree', 0.7672333121299744),
 ('rusk', 0.7649476528167725),
 ('rusks', 0.7439418435096741),
 ('finger_foods', 0.7421448230743408),
 ('soya_milk', 0.734347403049469),
 ('weetabix', 0.7281665802001953),
 ('fruit_puree', 0.7269812822341919),
 ('whole_milk', 0.7148292660713196),
 ('finger_food', 0.7144719958305359),
 ('pureed_veg', 0.7058597207069397),
 ('yoghurt', 0.7026206254959106),
 ('pureed_fruit', 0.6998013854026794),
 ('mashed_banana', 0.6846105456352234),
 ('nutramigen', 0.683875560760498),
 ('yogurt', 0.6820566654205322),
 ('neocate', 0.6790235042572021),
 ('solids', 0.6737068891525269),
 ('pures', 0.6699245572090149),
 ('gaviscon', 0.669344425201416),
 ('ready_brek', 0.6686376929283142),
 ('cow_gate', 0.6685115694999695),
 ('cows_milk', 0.6589570641517639),
 ('cow_and_gate', 0.6586804986000061),
 ('solid_food', 0.6569234132766724),
 ('fruit_pure', 0.6556069254875183),
 (

In [ ]:
model.wv.most_similar(['fruit','puree', 'infant', 'contamination'], negative=['recipe', 'meat'], topn=70)

In [ ]:
model.wv.most_similar(['baby_food'], negative=['recipe', 'diy'], topn=70)

In [ ]:
model.wv.most_similar(['baby_food'], negative=['homemade'], topn=70)

In [ ]:
model.wv.most_similar(['yoghurt'], topn=70)

In [ ]:
model.wv.most_similar(['baby_food', 'premade'], topn=70)

In [ ]:
model.wv.most_similar(['baby_food', 'premade'], negative=['homemade'], topn=70)

In [ ]:
model.wv.most_similar(['aldi_organic'], topn=70)

## Get words to augment our term search algorithm


In [69]:
model.wv.most_similar(['fruit'], topn=140)


[('cereal', 0.8440166711807251),
 ('yoghurt', 0.8279641270637512),
 ('yogurt', 0.8221521973609924),
 ('cheese', 0.8083785176277161),
 ('fresh_fruit', 0.8002566695213318),
 ('bread', 0.7951725721359253),
 ('apples', 0.7901284098625183),
 ('snacks', 0.7894400954246521),
 ('vegetables', 0.7883036136627197),
 ('fruit_juice', 0.7876205444335938),
 ('fruitveg', 0.7844418287277222),
 ('bananas', 0.7837749123573303),
 ('veg', 0.7831305265426636),
 ('crackers', 0.7814939618110657),
 ('peanut_butter', 0.7775682806968689),
 ('raisins', 0.7773728370666504),
 ('dried_fruit', 0.7765558362007141),
 ('yogurts', 0.7695724964141846),
 ('biscuits', 0.7689782977104187),
 ('berries', 0.7672105431556702),
 ('cereals', 0.7638691663742065),
 ('blueberries', 0.7603920698165894),
 ('breadsticks', 0.7589836120605469),
 ('veggies', 0.7583255171775818),
 ('porridge', 0.7575973868370056),
 ('fruits', 0.7572048306465149),
 ('rice_cakes', 0.7527785301208496),
 ('strawberries', 0.7521820068359375),
 ('crisps', 0.75153

In [ ]:
model.wv.most_similar(['fruit', puree], topn=70)

In [66]:
model.wv.most_similar('vegetable', topn=70)

[('corn', 0.8947718143463135),
 ('brown_rice', 0.8753851056098938),
 ('spinach', 0.8749935626983643),
 ('pesto', 0.8736596703529358),
 ('potato', 0.8731688857078552),
 ('dried_fruit', 0.8719448447227478),
 ('curries', 0.8702594041824341),
 ('lentil', 0.8690915703773499),
 ('wholemeal', 0.8689807653427124),
 ('cous_cous', 0.8644014596939087),
 ('tofu', 0.863556981086731),
 ('herb', 0.861121654510498),
 ('parmesan', 0.859762966632843),
 ('lentils', 0.859761655330658),
 ('coconut', 0.8594407439231873),
 ('cauliflower', 0.859309196472168),
 ('butternut_squash', 0.8591725826263428),
 ('salmon', 0.8575754761695862),
 ('stews', 0.8574146032333374),
 ('cream_cheese', 0.8569837808609009),
 ('mashed_potatoes', 0.8569060564041138),
 ('breads', 0.856400191783905),
 ('beetroot', 0.8560555577278137),
 ('pulses', 0.8560413122177124),
 ('tomato_sauce', 0.8537781834602356),
 ('couscous', 0.8536327481269836),
 ('tinned', 0.8532856702804565),
 ('mango', 0.8532373905181885),
 ('pasta_sauce', 0.85322201251

In [71]:
model.wv.most_similar('formula', topn=200)

[('milk', 0.8190065622329712),
 ('formula_milk', 0.810473620891571),
 ('breast_milk', 0.7841083407402039),
 ('cows_milk', 0.7440943717956543),
 ('bottles', 0.7322356700897217),
 ('breastmilk', 0.7247856855392456),
 ('follow_on_milk', 0.7181817293167114),
 ('aptamil', 0.711974024772644),
 ('powder', 0.6842058897018433),
 ('bottle', 0.6828750371932983),
 ('comfort_milk', 0.6825845837593079),
 ('expressed_milk', 0.670891284942627),
 ('milks', 0.6656394004821777),
 ('nutramigen', 0.665075421333313),
 ('sma', 0.6639872193336487),
 ('soya_milk', 0.6623433232307434),
 ('goats_milk', 0.6604939103126526),
 ('neocate', 0.6476559042930603),
 ('gaviscon', 0.6382917165756226),
 ('solid_food', 0.6371954679489136),
 ('solids', 0.6354448199272156),
 ('tap_water', 0.6232985854148865),
 ('feeds', 0.613257884979248),
 ('bm', 0.6118237972259521),
 ('rusks', 0.6111374497413635),
 ('cow_and_gate', 0.608920156955719),
 ('breast', 0.6077016592025757),
 ('powdered_milk', 0.6061331629753113),
 ('normal_formula'

In [90]:
model.wv.most_similar('aptamil', topn=200)

[('sma', 0.929912805557251),
 ('cow_and_gate', 0.9218206405639648),
 ('c_g', 0.8849695324897766),
 ('aptimil', 0.8592697978019714),
 ('cow_gate', 0.855187714099884),
 ('aptimal', 0.8534468412399292),
 ('nutramigen', 0.8393101096153259),
 ('hipp_organic', 0.8291254043579102),
 ('neocate', 0.8227742910385132),
 ('comfort_milk', 0.8132438063621521),
 ('hipp', 0.7599729895591736),
 ('follow_on_milk', 0.7591363191604614),
 ('sma_gold', 0.7577837705612183),
 ('reflux_milk', 0.7435489892959595),
 ('pepti', 0.7399816513061523),
 ('soya_formula', 0.7281014919281006),
 ('goats_milk', 0.7221401333808899),
 ('soya_milk', 0.7216423153877258),
 ('anti_reflux', 0.7182803153991699),
 ('formula', 0.7119741439819336),
 ('omeprazole', 0.6987064480781555),
 ('gaviscon', 0.6915204524993896),
 ('aptamil_comfort', 0.6914382576942444),
 ('milks', 0.6904162764549255),
 ('infant_milk', 0.6892383694648743),
 ('aptamil_pepti', 0.6886734962463379),
 ('hungry_baby', 0.6800727844238281),
 ('aptimel', 0.6711947321891

In [72]:
model.wv.most_similar(['brand','formula'], topn=70)

[('formula_milk', 0.8032941818237305),
 ('milks', 0.7653893232345581),
 ('follow_on_milk', 0.7413113117218018),
 ('aptamil', 0.740128219127655),
 ('product', 0.7115566730499268),
 ('breast_milk', 0.7067896127700806),
 ('goats_milk', 0.6993610858917236),
 ('soya_milk', 0.6881943941116333),
 ('formulas', 0.6871488690376282),
 ('milk', 0.6739535331726074),
 ('sma', 0.6722294688224792),
 ('breastmilk', 0.6683167219161987),
 ('cow_and_gate', 0.6598837375640869),
 ('powdered_milk', 0.6593323945999146),
 ('cow_gate', 0.6549218893051147),
 ('cartons', 0.6480847001075745),
 ('powder', 0.6475186944007874),
 ('nutramigen', 0.6474807262420654),
 ('brands', 0.6469166278839111),
 ('version', 0.6441795825958252),
 ('cows_milk', 0.6404333114624023),
 ('aptimil', 0.6400396823883057),
 ('rice_milk', 0.6314616203308105),
 ('rusks', 0.6294333338737488),
 ('baby_food', 0.6294211149215698),
 ('milk_powder', 0.6286181211471558),
 ('hipp_organic', 0.6284900903701782),
 ('products', 0.6273252964019775),
 ('bot

In [75]:
model.wv.most_similar(['hipp'], topn=70)

[('hipp_organic', 0.8754072785377502),
 ('cow_gate', 0.8602960109710693),
 ('cow_and_gate', 0.8441417217254639),
 ('ellas_kitchen', 0.8299736976623535),
 ('c_g', 0.7919853329658508),
 ('fruit_pots', 0.7881747484207153),
 ('organix', 0.7611843943595886),
 ('aptamil', 0.7599730491638184),
 ('aptimal', 0.7534179091453552),
 ('sma', 0.751855194568634),
 ('tescos_own', 0.7503743767738342),
 ('baby_porridge', 0.739824652671814),
 ('mamia', 0.7295929789543152),
 ('pouches', 0.7292193174362183),
 ('aptimil', 0.7239006757736206),
 ('heinz', 0.721105694770813),
 ('ellas', 0.7147186994552612),
 ('plum_baby', 0.7018450498580933),
 ('porridges', 0.7011147737503052),
 ('powdered', 0.6981130242347717),
 ('fromage_frais', 0.692169725894928),
 ('creamy_porridge', 0.6832783222198486),
 ('asda_own', 0.6830801367759705),
 ('nuk', 0.6795035600662231),
 ('lacto_free', 0.6791443228721619),
 ('ready_made', 0.6789427995681763),
 ('tesco_own', 0.67740797996521),
 ('closer_to_nature', 0.6753034591674805),
 ('fru

In [87]:
model.wv.most_similar(['nutramigen'], topn=70)

[('neocate', 0.8744420409202576),
 ('aptamil', 0.8393102288246155),
 ('comfort_milk', 0.8225110769271851),
 ('sma', 0.8214061260223389),
 ('pepti', 0.8114845156669617),
 ('omeprazole', 0.8067999482154846),
 ('gaviscon', 0.8061847686767578),
 ('movicol', 0.7930401563644409),
 ('lactulose', 0.7884836792945862),
 ('soya_formula', 0.7876734733581543),
 ('cow_and_gate', 0.7847892045974731),
 ('ranitidine', 0.7739846110343933),
 ('c_g', 0.7733110785484314),
 ('soya_milk', 0.7727905511856079),
 ('aptamil_pepti', 0.7695236802101135),
 ('cow_gate', 0.7588754892349243),
 ('reflux_milk', 0.7561953663825989),
 ('goats_milk', 0.7538076639175415),
 ('colief', 0.7502973675727844),
 ('aptimil', 0.7492396831512451),
 ('infant_gaviscon', 0.744446337223053),
 ('wysoy', 0.7407585978507996),
 ('sma_gold', 0.7368431091308594),
 ('prescription_milk', 0.7262526750564575),
 ('follow_on_milk', 0.7257347702980042),
 ('cows_milk', 0.7248399257659912),
 ('aptimal', 0.7194741368293762),
 ('hipp_organic', 0.71240389

In [ ]:
model.wv.most_similar(['powder'], topn=70)

In [89]:
model.wv.most_similar(['c_g'], topn=70)

[('cow_and_gate', 0.9008226990699768),
 ('aptamil', 0.8849696516990662),
 ('sma', 0.861832320690155),
 ('aptimal', 0.8530339598655701),
 ('hipp_organic', 0.8417338132858276),
 ('cow_gate', 0.8401004672050476),
 ('aptimil', 0.8116588592529297),
 ('sma_gold', 0.7940136790275574),
 ('hipp', 0.7919853925704956),
 ('pepti', 0.7776606678962708),
 ('nutramigen', 0.7733111381530762),
 ('neocate', 0.7669175863265991),
 ('comfort_milk', 0.7590739130973816),
 ('anti_reflux', 0.7484025955200195),
 ('aptimel', 0.7393959760665894),
 ('aptamil_comfort', 0.7282416224479675),
 ('soya_formula', 0.7279740571975708),
 ('aptamil_first', 0.7265149354934692),
 ('infant_milk', 0.7230526208877563),
 ('reflux_milk', 0.7105595469474792),
 ('omeprazole', 0.6968874335289001),
 ('infasoy', 0.6912573575973511),
 ('soya_milk', 0.6889443397521973),
 ('aptamil_pepti', 0.6886391043663025),
 ('prescription_milk', 0.6866732239723206),
 ('pepti_junior', 0.6821683645248413),
 ('hungry_baby', 0.680678129196167),
 ('althera',

In [77]:
model.wv.most_similar(['baby_rice'], topn=70)

[('baby_porridge', 0.8808053731918335),
 ('porridge', 0.8104282021522522),
 ('purees', 0.7915183305740356),
 ('puree', 0.7672333121299744),
 ('rusk', 0.7649476528167725),
 ('rusks', 0.7439418435096741),
 ('finger_foods', 0.7421448230743408),
 ('soya_milk', 0.734347403049469),
 ('weetabix', 0.7281665802001953),
 ('fruit_puree', 0.7269812822341919),
 ('whole_milk', 0.7148292660713196),
 ('finger_food', 0.7144719958305359),
 ('pureed_veg', 0.7058597207069397),
 ('yoghurt', 0.7026206254959106),
 ('pureed_fruit', 0.6998013854026794),
 ('mashed_banana', 0.6846105456352234),
 ('nutramigen', 0.683875560760498),
 ('yogurt', 0.6820566654205322),
 ('neocate', 0.6790235042572021),
 ('solids', 0.6737068891525269),
 ('pures', 0.6699245572090149),
 ('gaviscon', 0.669344425201416),
 ('ready_brek', 0.6686376929283142),
 ('cow_gate', 0.6685115694999695),
 ('cows_milk', 0.6589570641517639),
 ('cow_and_gate', 0.6586804986000061),
 ('solid_food', 0.6569234132766724),
 ('fruit_pure', 0.6556069254875183),
 (

In [91]:
model.wv.most_similar(['mamia'], topn=70)

[('asda_own', 0.8009127378463745),
 ('tescos_own', 0.793470561504364),
 ('supermarket_own', 0.7925098538398743),
 ('asdas_own', 0.7751727104187012),
 ('morrisons_own', 0.7690659761428833),
 ('hipp_organic', 0.768993616104126),
 ('powdered', 0.7623755931854248),
 ('plum_baby', 0.7603628039360046),
 ('shops_own', 0.7573117017745972),
 ('boots_own', 0.7567915320396423),
 ('tesco_own', 0.7566094994544983),
 ('sainsburys_own', 0.7547160387039185),
 ('supermarkets_own', 0.7536826729774475),
 ('ellas', 0.7535243034362793),
 ('readymade', 0.7497919797897339),
 ('cow_gate', 0.7472209334373474),
 ('tesco_value', 0.745101809501648),
 ('uht_milk', 0.7428492903709412),
 ('sainsburys_basics', 0.7394733428955078),
 ('little_angels', 0.7385008931159973),
 ('porridges', 0.7358334064483643),
 ('pre_made', 0.7355183362960815),
 ('ready_made', 0.7352178692817688),
 ('heinz', 0.7338820695877075),
 ('hipp', 0.7295929193496704),
 ('nuk', 0.7284761071205139),
 ('premium', 0.727486789226532),
 ('dishwasher_tab

# Examine \_contains and other possible rankings


In [80]:
contains = [item for item in bigram_transformer.export_phrases().items() if '_contains' in item[0]]

In [81]:
contains

[('it_contains', 2.9932579706858733),
 ('also_contains', 2.274479707268448),
 ('formula_contains', 2.562611533222873),
 ('milk_contains', 10.141872135203322),
 ('texture_contains', 17.37389604964793),
 ('breastmilk_contains', 21.103254199122485),
 ('which_contains', 4.037811834569131),
 ('actually_contains', 3.4413424054133044),
 ('vaccine_contains', 5.9111905551457395),
 ('rusk_contains', 6.7984810629057115)]

In [82]:
model.wv.most_similar(['bpa'], topn=70)

[('arsenic', 0.8115674257278442),
 ('oestrogen', 0.7964176535606384),
 ('mercury', 0.7922999262809753),
 ('sodium', 0.7916605472564697),
 ('fluoride', 0.7734161019325256),
 ('parabens', 0.7723528742790222),
 ('aluminium', 0.7576263546943665),
 ('toxicity', 0.7467406988143921),
 ('plastics', 0.7460504770278931),
 ('tannin', 0.7447837591171265),
 ('fatty_acids', 0.739966869354248),
 ('chemicals', 0.7391164898872375),
 ('amino_acids', 0.7389072179794312),
 ('potassium', 0.7369123697280884),
 ('aspartame', 0.7349771857261658),
 ('chemical', 0.7339134812355042),
 ('preservative', 0.7317711114883423),
 ('cancer_causing', 0.7314833998680115),
 ('refined_sugar', 0.7294902801513672),
 ('additives', 0.7273610234260559),
 ('pesticides', 0.726372480392456),
 ('minerals', 0.7217450141906738),
 ('molecules', 0.7212138772010803),
 ('cosmetic', 0.7207120656967163),
 ('infant_formula', 0.7197909355163574),
 ('naturally_occurring', 0.7191856503486633),
 ('aluminum', 0.7182139754295349),
 ('nasties', 0.7